In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install paddlenlp==2.4.2 -i https://pypi.tuna.tsinghua.edu.cn/simple -t /home/aistudio/external-libraries
!pip install datasets -t /home/aistudio/external-libraries
!pip install rouge -t /home/aistudio/external-libraries

mkdir: 无法创建目录"/home/aistudio/external-libraries": 文件已存在
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.8 MB ? eta -:--:--━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.8 MB 8.7 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━ 0.9/1.8 MB 13.5 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 1.5/1.8 MB 15.1 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.9 MB/s eta 0:00:00
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/101.0 kB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 3.7 MB/s eta 0:00:00
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/86.9 kB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 14.3 MB/s eta 0:00:00
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/19.2 MB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/19.2 MB 7.2 MB/s eta 0:00:03━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/19.2 MB 10.1 

# 准备引用

In [1]:
import sys 
sys.path.append('/home/aistudio/external-libraries')
sys.path.append('/home/aistudio/work/aigc-pkg-paddle')
import json

# 准备数据转换代码

In [2]:
import paddlenlp
paddlenlp.__version__

'2.4.2'

In [3]:
from paddlenlp.datasets import advertisegen

In [4]:
adv_data = advertisegen.AdvertiseGen(name='advertisegen')

In [5]:
avdertisegen_files_path = ['/home/aistudio/work/AdvertiseGen/train.json', '/home/aistudio/work/AdvertiseGen/dev.json']
advertisegen_splits = ['train', 'dev']
train_dataset, dev_dataset = adv_data.read_datasets(advertisegen_splits, avdertisegen_files_path)

In [6]:
# !export PPNLP_HOME='/home/aistudio/external-libraries'

In [7]:
from paddlenlp.transformers import PegasusForConditionalGeneration, PegasusChineseTokenizer

# 初始化分词器
tokenizer = PegasusChineseTokenizer.from_pretrained('IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese')

# 初始化模型，也可以选择IDEA-CCNL/Randeng-Pegasus-523M-Summary-Chinese
model = PegasusForConditionalGeneration.from_pretrained('IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese')

[2023-01-19 11:52:43,157] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/community/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/vocab.txt and saved to /home/aistudio/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese
[2023-01-19 11:52:43,159] [    INFO] - Downloading vocab.txt from https://bj.bcebos.com/paddlenlp/models/community/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/vocab.txt
100%|██████████| 365k/365k [00:00<00:00, 9.33MB/s]
[2023-01-19 11:52:43,373] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/community/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/added_tokens.json and saved to /home/aistudio/.paddlenlp/models/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese
[2023-01-19 11:52:43,375] [    INFO] - Downloading added_tokens.json from https://bj.bcebos.com/paddlenlp/models/community/IDEA-CCNL/Randeng-Pegasus-238M-Summary-Chinese/added_tokens.json
100%|██████████| 2.00/2.00 [00:00<00:00, 1.54kB/s]
[2023-01-19 11:52:43,460] [   

In [8]:
from importlib import reload
from aigc_pkg import prepare
reload(prepare)

<module 'aigc_pkg.prepare' from '/home/aistudio/work/aigc-pkg-paddle/aigc_pkg/prepare.py'>

In [9]:
# 原始字段需要移除
remove_columns = ['source', 'target', 'id']
# 文本的最大长度
max_source_length = 128
# 摘要的最大长度
max_target_length = 64

model_set = prepare.ModelSet(tokenizer, model)
prepare_config = prepare.AbstractPrepareConfig(text_column=remove_columns[0], target_column=remove_columns[1],
    max_source_length=max_source_length, max_target_length=max_target_length, min_target_length=0, batch_size=12)

prepare_tool = prepare.AbstractGenerationPrapare(model_set, prepare_config)

In [11]:
train_dataset, dev_dataset = prepare_tool.get_dataset(train_dataset, dev_dataset, remove_columns)

In [12]:
train_data_loader, dev_data_loader = prepare_tool.get_dataloader(train_dataset, dev_dataset)

In [13]:
# 输出训练集的前 3 条样本
for idx, example in enumerate(dev_dataset):
    if idx < 3:
        print(example) 

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.800 seconds.
Prefix dict has been built successfully.


{'input_ids': [27381, 108, 6809, 115, 21641, 108, 3115, 312, 1591, 115, 34263, 108, 25808, 115, 34303, 108, 27296, 115, 13777, 108, 11018, 115, 4511, 22039, 108, 14380, 115, 4511, 22377, 108, 3519, 2768, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27296, 3968, 200, 27283, 3399, 24809, 14380, 5661, 25808, 3399, 4511, 4853, 11559, 3397, 2131, 179, 4511, 4853, 14465, 2335, 469, 2245, 3519, 2768, 30561, 5661, 18839, 709, 4667, 245, 860, 5661, 14189, 6226, 32455, 26196, 179, 4511, 31555, 4030, 1218, 2335, 31427, 11018, 30152, 5661, 7524, 38695, 5661, 17741, 10945, 20847, 179, 1]}
{'input_ids': [27381, 108, 4544, 115, 21641, 108, 32977, 115, 34263, 108, 27696, 115, 34303, 108, 14317, 115, 34303, 108, 20406, 115, 34303, 108, 27296, 115, 13777, 108, 22074, 115, 13777, 108, 27696, 115, 13777, 108, 14317, 115, 4544, 1138, 108, 28704, 4544, 115, 4544, 5219, 108, 42413, 115, 4544, 5406, 1138, 108, 702, 34753,

# 训练模型

In [14]:
from aigc_pkg import generator
reload(generator)

<module 'aigc_pkg.generator' from '/home/aistudio/work/aigc-pkg-paddle/aigc_pkg/generator.py'>

In [17]:
gen_config = generator.AbstractGeneratorConfig(
    # 学习率预热比例
    warmup = 0.02,
    # 学习率
    learning_rate = 5e-5,
    # 训练轮次
    num_epochs = 3,
    # AdamW优化器参数epsilon
    adam_epsilon = 1e-6,
    # AdamW优化器参数weight_decay
    weight_decay=0.01,
    # 训练中，每个log_steps打印一次日志
    log_steps = 100,
    # 训练中，每隔eval_steps进行一次模型评估
    eval_steps = 1000,
    # 摘要的最小长度
    min_target_length = 0,
    # 训练模型保存路径
    output_dir = 'checkpoints',
    # 解码beam size
    num_beams = 4,
    log_dir = 'visualdl_log_dir'
)

gen = generator.AbstractGenerator(train_data_loader, dev_data_loader, model_set, prepare_config, gen_config)

In [ ]:
# 调用模型训练
gen.train()

[2023-01-19 11:55:41,914] [    INFO] - global step 100/28650, epoch: 0, batch: 99, rank_id: 0, loss: 3.479999, lr: 0.0000087260, speed: 6.3097 step/s
[2023-01-19 11:55:56,675] [    INFO] - global step 200/28650, epoch: 0, batch: 199, rank_id: 0, loss: 3.536752, lr: 0.0000174520, speed: 6.7791 step/s
[2023-01-19 11:56:11,356] [    INFO] - global step 300/28650, epoch: 0, batch: 299, rank_id: 0, loss: 3.450709, lr: 0.0000261780, speed: 6.8162 step/s
[2023-01-19 11:56:26,306] [    INFO] - global step 400/28650, epoch: 0, batch: 399, rank_id: 0, loss: 3.693245, lr: 0.0000349040, speed: 6.6937 step/s
[2023-01-19 11:56:40,969] [    INFO] - global step 500/28650, epoch: 0, batch: 499, rank_id: 0, loss: 3.355099, lr: 0.0000436300, speed: 6.8246 step/s
[2023-01-19 11:56:55,707] [    INFO] - global step 600/28650, epoch: 0, batch: 599, rank_id: 0, loss: 2.931969, lr: 0.0000499519, speed: 6.7899 step/s
[2023-01-19 11:57:10,438] [    INFO] - global step 700/28650, epoch: 0, batch: 699, rank_id: 0,


***************
The auto evaluation result is:
rouge-1: 38.88
rouge-2: 15.75
rouge-L: 26.42
BLEU-4: 8.52


[2023-01-19 12:00:35,466] [    INFO] - eval done total : 159.7616035938263 s
[2023-01-19 12:00:39,153] [    INFO] - tokenizer config file saved in checkpoints/tokenizer_config.json
[2023-01-19 12:00:39,156] [    INFO] - Special tokens file saved in checkpoints/special_tokens_map.json
[2023-01-19 12:00:54,014] [    INFO] - global step 1100/28650, epoch: 0, batch: 1099, rank_id: 0, loss: 3.006933, lr: 0.0000490615, speed: 0.5608 step/s
[2023-01-19 12:01:08,886] [    INFO] - global step 1200/28650, epoch: 0, batch: 1199, rank_id: 0, loss: 3.212152, lr: 0.0000488834, speed: 6.7287 step/s
[2023-01-19 12:01:23,934] [    INFO] - global step 1300/28650, epoch: 0, batch: 1299, rank_id: 0, loss: 3.333295, lr: 0.0000487053, speed: 6.6496 step/s
[2023-01-19 12:01:38,732] [    INFO] - global step 1400/28650, epoch: 0, batch: 1399, rank_id: 0, loss: 2.908715, lr: 0.0000485273, speed: 6.7626 step/s
[2023-01-19 12:01:53,694] [    INFO] - global step 1500/28650, epoch: 0, batch: 1499, rank_id: 0, loss:


***************
The auto evaluation result is:
rouge-1: 39.64
rouge-2: 16.72
rouge-L: 27.24
BLEU-4: 8.86


[2023-01-19 12:05:48,877] [    INFO] - eval done total : 160.26383233070374 s
[2023-01-19 12:06:02,524] [    INFO] - tokenizer config file saved in checkpoints/tokenizer_config.json
[2023-01-19 12:06:02,528] [    INFO] - Special tokens file saved in checkpoints/special_tokens_map.json
[2023-01-19 12:06:17,999] [    INFO] - global step 2100/28650, epoch: 0, batch: 2099, rank_id: 0, loss: 3.295329, lr: 0.0000472807, speed: 0.5280 step/s
[2023-01-19 12:06:32,742] [    INFO] - global step 2200/28650, epoch: 0, batch: 2199, rank_id: 0, loss: 3.268803, lr: 0.0000471026, speed: 6.7872 step/s
[2023-01-19 12:06:47,626] [    INFO] - global step 2300/28650, epoch: 0, batch: 2299, rank_id: 0, loss: 3.222262, lr: 0.0000469245, speed: 6.7231 step/s
[2023-01-19 12:07:02,438] [    INFO] - global step 2400/28650, epoch: 0, batch: 2399, rank_id: 0, loss: 2.956820, lr: 0.0000467464, speed: 6.7558 step/s
[2023-01-19 12:07:17,272] [    INFO] - global step 2500/28650, epoch: 0, batch: 2499, rank_id: 0, loss

# 模型预测

In [ ]:
# 加载训练好的模型
model = PegasusForConditionalGeneration.from_pretrained('checkpoints')
model.eval()
tokenizer = PegasusChineseTokenizer.from_pretrained('checkpoints')

gen.model_set.model = model
gen.model_set.tokenizer = tokenizer

In [ ]:
# 推理
# TODO 换成advertisegen的数据
text = '类型#裤*版型#宽松*风格#性感*图案#线条*裤型#阔腿裤'
gen.infer(text, text_column='source', decode_strategy='beam_search')